In [172]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import QFT
from qiskit.visualization import array_to_latex

import matplotlib.pyplot as plt

import numpy as np

In [226]:
# implements a controlled phase gate

def cp_gate(theta):
    quantum_register = QuantumRegister(2, 'x')
    quantum_circuit = QuantumCircuit(quantum_register, name = 'cp(pi*' + str(round(theta/np.pi, 3)) + ')')

    quantum_circuit.cx(quantum_register[0], quantum_register[1])
    quantum_circuit.p(-theta/2,quantum_register[1])
    quantum_circuit.cx(quantum_register[0], quantum_register[1])
    quantum_circuit.p(theta/2,quantum_register[:])
    
    return quantum_circuit.to_gate()

In [227]:
# implements 2- and 3-controlled phase gates using controlled phase gates
# decomposition of qiskit's implementation of multi-controlled phase gates

def ccp_gate(theta): 
    pos_cp_gate = cp_gate(theta/2)
    neg_cp_gate = cp_gate(-theta/2)

    quantum_register = QuantumRegister(3, 'x')
    quantum_circuit = QuantumCircuit(quantum_register, name = 'ccp(pi*' + str(round(theta/np.pi, 3)) + ')')

    quantum_circuit.compose(pos_cp_gate, qubits = quantum_register[1:], inplace=True)
    
    quantum_circuit.cx(quantum_register[1], quantum_register[0])
    quantum_circuit.compose(neg_cp_gate, qubits = [quantum_register[0], quantum_register[2]], inplace=True)
    quantum_circuit.cx(quantum_register[1], quantum_register[0])
    quantum_circuit.compose(pos_cp_gate, qubits = [quantum_register[0], quantum_register[2]], inplace=True)

    return quantum_circuit.to_gate()

def cccp_gate(theta):
    pos_cp_gate = cp_gate(theta/4)
    neg_cp_gate = cp_gate(-theta/4)

    quantum_register = QuantumRegister(4, 'x')
    quantum_circuit = QuantumCircuit(quantum_register, name = 'cccp(pi*' + str(round(theta/np.pi, 3)) + ')')

    quantum_circuit.compose(pos_cp_gate, qubits = quantum_register[2:], inplace=True)
    
    quantum_circuit.cx(quantum_register[2], quantum_register[1])
    quantum_circuit.compose(neg_cp_gate, qubits = [quantum_register[1], quantum_register[3]], inplace=True)
    quantum_circuit.cx(quantum_register[2], quantum_register[1])
    quantum_circuit.compose(pos_cp_gate, qubits = [quantum_register[1], quantum_register[3]], inplace=True)

    quantum_circuit.cx(quantum_register[1], quantum_register[0])
    quantum_circuit.compose(neg_cp_gate, qubits = [quantum_register[0], quantum_register[3]], inplace=True)
    quantum_circuit.cx(quantum_register[2], quantum_register[0])
    quantum_circuit.compose(pos_cp_gate, qubits = [quantum_register[0], quantum_register[3]], inplace=True)

    quantum_circuit.cx(quantum_register[1], quantum_register[0])
    quantum_circuit.compose(neg_cp_gate, qubits = [quantum_register[0], quantum_register[3]], inplace=True)
    quantum_circuit.cx(quantum_register[2], quantum_register[0])
    quantum_circuit.compose(pos_cp_gate, qubits = [quantum_register[0], quantum_register[3]], inplace=True)
    return quantum_circuit.to_gate()

In [228]:
# # implements the quantum Fourier transform
# # implementation following the algorithm described in Draper's paper on quantum addition
# # https://arxiv.org/abs/quant-ph/0008033

def qft(n):
    
    quantum_register = QuantumRegister(size = n, name='x')
    quantum_circuit = QuantumCircuit(quantum_register, name = 'QFT' + str(n))

    for i in range(n): 
        quantum_circuit.h(quantum_register[i])
        for j in range(i + 1, n):
            quantum_circuit.compose(cp_gate(np.pi/(2**(j - i))), qubits = [quantum_register[j], quantum_register[i]], inplace=True)
    
    # swaps the qubits to put them in the correct order
    for i in range(n // 2):
        quantum_circuit.swap(quantum_register[i], quantum_register[n - i - 1])

    return quantum_circuit.to_gate()

# implements the inverse quantum Fourier transform using phase gates and cx gates
def qft_inverse(n):
    quantum_register = QuantumRegister(size = n, name='x')
    quantum_circuit = QuantumCircuit(quantum_register, name = 'IOFT' + str(n))
    
    for i in range(n // 2):
        quantum_circuit.swap(quantum_register[i], quantum_register[n - i - 1])
    
    for i in reversed(range(n)): 
        for j in reversed(range(i + 1, n)):
            quantum_circuit.compose(cp_gate(-np.pi/(2**(j - i))), qubits = [quantum_register[i], quantum_register[j]], inplace=True)
        quantum_circuit.h(quantum_register[i])

    return quantum_circuit.to_gate()


In [ ]:
# implements modified Draper algorithm for quantum addition and multiplication
# using the quantum Fourier transform and inverse quantum Fourier transform
# the algorithm is used to (depending on z) add or multiply two d-bit numbers x and y and add the result to the output register w
def QCalc(d):
    # x and y are the two numbers to be added or multiplied
    # z is the operation to be performed (0 for addition, 1 for multiplication)
    # w/output_register is the result of the operation
    x_register = QuantumRegister(size = d, name = 'x')
    y_register = QuantumRegister(size = d, name = 'y')
    z_register = QuantumRegister(size = 1, name = 'z')
    output_register = QuantumRegister(size = d, name = 'w')

    quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register)

    # convert output register to phase space
    quantum_circuit.compose(qft(d), qubits = output_register, inplace = True)

    # decomposes the scaling |x,y,z,w> \mapsto e^{2*pi*i*(xyzw + (x+y)(z-1)w)/2^d}*|x,y,z,w> into a series of 2- and 3-controlled phase gates
    # for z = 0, the operation recovers addition, for z = 1, the operation recovers multiplication
    # writing x = 2^{d-1}*x_{d-1} + 2^{d-2}*x_{d-2} + ... + 2^0*x_0, y, w analogous
    
    # multiplication part: runs for z = 1
    # decompose into phase shifts by (2*pi*i/2^d) * 2^t*x_i*y_j*z*w_k for t = i+j+k, 0 <= t <= d-1 (since operations are modulo 2^d)
    for t in range(d):
        phase_shift = cccp_gate(np.pi/(2**(d - 1 - t)))
        for i in range(t + 1):
            for j in range(t - i + 1):
                k = t - i - j
                quantum_circuit.compose(phase_shift, qubits = [x_register[d - 1 - i], y_register[d - 1 - j], z_register[0], output_register[d - 1 - k]], inplace=True)
    

    # addition part: runs for z = 0
    # decompose into phase shifts by (2*pi*i/2^d) * 2^t*x_i*z*w_j for t = i+j, 0 <= t <= d-1, or (2*pi*i/2^d) * 2^t*y_i*z*w_j
    quantum_circuit.x(z_register[0])
    for t in range(d):
        phase_shift = ccp_gate(np.pi/(2**(d - 1 - t)))
        for i in range(t + 1):
            j = t - i
            quantum_circuit.compose(phase_shift, qubits = [x_register[d - 1 - i], z_register[0], output_register[d - 1 - j]], inplace=True)
            quantum_circuit.compose(phase_shift, qubits = [y_register[d - 1 - i], z_register[0], output_register[d - 1 - j]], inplace=True)
    quantum_circuit.x(z_register[0])

    # convert output register back to computational space
    quantum_circuit.compose(qft_inverse(d), qubits = output_register, inplace = True)
    return quantum_circuit

Note that for the benchmarking, the qubits should be read backwards, i.e. if we are working with QCalc(3) as in the first example below, then $$|0001001001\rangle $$ should be interpreted as $$| 100 \rangle | 100 \rangle |1\rangle |000 \rangle.$$

In [221]:
# benchmarking the quantum calculator

x_register = QuantumRegister(3, 'x')
y_register = QuantumRegister(3, 'y')
z_register = QuantumRegister(1, 'z')
output_register = QuantumRegister(3, 'w')

quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register)

quantum_calculator_gate = QCalc(3).to_gate()
quantum_circuit.x([x_register[0], y_register[0], z_register[0]])
quantum_circuit.compose(quantum_calculator_gate, qubits = x_register[:] + y_register[:] + z_register[:] + output_register[:], inplace = True)

Statevector(quantum_circuit).draw('latex')


<IPython.core.display.Latex object>

$$4\times 4 = 0 \text{ mod } 8$$

In [194]:
# benchmarking the quantum calculator

x_register = QuantumRegister(3, 'x')
y_register = QuantumRegister(3, 'y')
z_register = QuantumRegister(1, 'z')
output_register = QuantumRegister(3, 'w')

quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register)

quantum_calculator_gate = QCalc(3).to_gate()
quantum_circuit.x([x_register[1], y_register[1]])
quantum_circuit.compose(quantum_calculator_gate, qubits = x_register[:] + y_register[:] + z_register[:] + output_register[:], inplace = True)

Statevector(quantum_circuit).draw('latex')


<IPython.core.display.Latex object>

$$2 + 2 = 4$$

In [231]:
# benchmarking the quantum calculator

x_register = QuantumRegister(3, 'x')
y_register = QuantumRegister(3, 'y')
z_register = QuantumRegister(1, 'z')
output_register = QuantumRegister(3, 'w')

quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register)

quantum_calculator_gate = QCalc(3).to_gate()
quantum_circuit.x([x_register[0] , y_register[1], y_register[2], z_register[0]])

quantum_circuit.compose(quantum_calculator_gate, qubits = x_register[:] + y_register[:] + z_register[:] + output_register[:], inplace = True)

Statevector(quantum_circuit).draw('latex')





<IPython.core.display.Latex object>

$$4 \times 3 = 12 \equiv 4 \text{ mod } 8$$

In [ ]:
# benchmarking the quantum calculator

x_register = QuantumRegister(4, 'x')
y_register = QuantumRegister(4, 'y')
z_register = QuantumRegister(1, 'z')
output_register = QuantumRegister(4, 'w')

quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register)

quantum_calculator_gate = QCalc(4).to_gate()
quantum_circuit.x([x_register[1], x_register[2], x_register[3] , y_register[2], y_register[3], z_register[0]])

quantum_circuit.compose(quantum_calculator_gate, qubits = x_register[:] + y_register[:] + z_register[:] + output_register[:], inplace = True)

Statevector(quantum_circuit).draw('latex')

<IPython.core.display.Latex object>

$$7 \times 3 = 21 \equiv 5 \text{ mod } 16$$

In [233]:
# benchmarking the quantum calculator

x_register = QuantumRegister(4, 'x')
y_register = QuantumRegister(4, 'y')
z_register = QuantumRegister(1, 'z')
output_register = QuantumRegister(4, 'w')

quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register)

quantum_calculator_gate = QCalc(4).to_gate()
quantum_circuit.x([x_register[1], x_register[2], x_register[3] , y_register[2], y_register[3]])

quantum_circuit.compose(quantum_calculator_gate, qubits = x_register[:] + y_register[:] + z_register[:] + output_register[:], inplace = True)

Statevector(quantum_circuit).draw('latex')

<IPython.core.display.Latex object>

$$7 + 3 = 10$$

In [235]:
# benchmarking the quantum calculator

x_register = QuantumRegister(5, 'x')
y_register = QuantumRegister(5, 'y')
z_register = QuantumRegister(1, 'z')
output_register = QuantumRegister(5, 'w')

quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register)

quantum_calculator_gate = QCalc(5).to_gate()
quantum_circuit.x([x_register[1], x_register[3], x_register[4] , y_register[3], y_register[4], z_register[0]] + output_register[:])

quantum_circuit.compose(quantum_calculator_gate, qubits = x_register[:] + y_register[:] + z_register[:] + output_register[:], inplace = True)

Statevector(quantum_circuit).draw('latex')

<IPython.core.display.Latex object>

Note the initialization of the output register as $11111$, which should be interpreted as $31$. So the calculation above is 
$$ 11 \times 3 + 31 = 64 \equiv 0 \text{ mod } 32. $$